<a href="https://colab.research.google.com/github/BuffaloManwich/CS5588-HW-1/blob/main/Week2_LLM_HandsOn_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 45-Minute Hands-On: LLMs with Hugging Face (Colab/Jupyter)

**Last updated:** 2025-09-01 05:29

## Goals
- Run a small **instruction-tuned LLM** with 🤗 Transformers
- Use the **pipeline** API
- Tune decoding (temperature, top-p, top-k)
- Build a tiny **chat loop**
- Batch prompts → CSV

In [40]:
# 1) Install dependencies
#!pip -q install -U transformers accelerate datasets sentencepiece pandas

In [41]:
# 2) Imports & device
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


## Model choice
We try **TinyLlama/TinyLlama-1.1B-Chat-v1.0** and fall back to **distilgpt2** if needed.

In [42]:
# 3) Load model
model_id = "SicariusSicariiStuff/Phi-3.5-mini-instruct_Uncensored"
fallback_model_id = "distilgpt2"

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            device_map="auto" if device == "cuda" else None
        )
        return tok, mdl, fallback_model_id

tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Loaded: SicariusSicariiStuff/Phi-3.5-mini-instruct_Uncensored


## Quickstart with `pipeline`

In [43]:
# 4) Text generation quickstart
gen = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "Explain what a Knowledge Graph is in healthcare, in 3 concise sentences."
out = gen(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
print(out)

Device set to use cuda:0


Explain what a Knowledge Graph is in healthcare, in 3 concise sentences.

A Knowledge Graph is a structured representation of information, where entities (such as patients, diseases, and treatments) and their relationships are stored as nodes and edges. In the healthcare domain, this allows for the efficient organization and retrieval of complex medical data, as well as the ability to perform advanced queries and analyses.

1. Knowledge Graphs enable the integration of diverse data sources, such as electronic health records (EHRs), medical literature, and clinical guidelines, into a unified and accessible platform. This facilitates


## Tokenization peek

In [44]:
# 5) Tokenization
text = "Large Language Models can draft emails and summarize clinical notes."
ids = tokenizer(text).input_ids
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))

Token count: 15
First 20 ids: [8218, 479, 17088, 3382, 1379, 508, 18195, 24609, 322, 19138, 675, 24899, 936, 11486, 29889]
Decoded: Large Language Models can draft emails and summarize clinical notes.


## Decoding controls (temperature/top-p/top-k)

In [45]:
# 6) Compare decoding
base_prompt = "Give 3 short tips for writing reproducible data science code:"
settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.4, "top_p": 0.95, "top_k": 50},
    {"temperature": 0.2, "top_p": 0.85, "top_k": 50},
    {"temperature": 0.2, "top_p": 0.95, "top_k": 70},
    {"temperature": 0.6, "top_p": 0.9, "top_k": 50},
    {"temperature": 0.8, "top_p": 0.9, "top_k": 50},
    {"temperature": 1.1, "top_p": 0.85, "top_k": 50},
]
for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(base_prompt, max_new_tokens=100, do_sample=True, temperature=s["temperature"], top_p=s["top_p"], top_k=s["top_k"], pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")


--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code:

1. Use relative paths instead of absolute paths: This will make your code more portable and easier to run on different machines.
2. Comment your code: This will help others understand what your code is doing and make it easier for you to debug if something goes wrong.
3. Keep your code organized: Use functions and modules to break down your code into smaller, more manageable pieces. This will make it easier to read and maintain.

Now, let's
(latency ~3.78s)

--- Variant 2 | temp=0.4 top_p=0.95 top_k=50 ---
Give 3 short tips for writing reproducible data science code:

1. Comment your code extensively: Make sure every line of your code has a clear explanation of what it does. This will help you remember what you wrote and make it easier for others to understand your work.

2. Use version control: Always use a version control system like Git to track changes to your code. This

## Minimal chat loop

In [46]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

Absolutely! Transfer learning is a powerful technique that has been successfully applied in various domains. For example, in computer vision, it has been used to improve the accuracy of object detection and recognition in images and videos. This is particularly useful
Oh, you're interested in the risks associated with fine-tuning small language models on tiny datasets? That's a fantastic question! Let me break it down for you.

First, let's discuss the concept of fine-tuning. Fine-tuning is a process in which a pre-trained model is adapted to a specific task by adjusting the weights of its parameters using a smaller, task-specific dataset. This is often done to improve the performance of the model on the target task. Now, when it comes to small language models (LLMs) on tiny datasets, there
I'm glad you asked! Let's dive into the two risks associated with fine-tuning small language models on tiny datasets and explore some mitigation strategies for each.

Risk 1: Overfitting
Overfitting

## Batch prompts → CSV

In [47]:
# 8) Batch prompts and save
import pandas as pd, time
prompts = [
    "Write a tweet (<=200 chars) about reproducible ML.",
    "One sentence: why eval metrics matter beyond accuracy.",
    "List 3 checks before deploying a model to production.",
    "Explain temperature vs. top-p to a PM."
]
rows = []
for p in prompts:
    t0 = time.time()
    out = gen(p, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9, pad_token_id=tokenizer.eos_token_id)[0]["generated_text"]
    rows.append({"prompt": p, "output": out, "latency_s": round(time.time()-t0, 2)})
df = pd.DataFrame(rows)
df

,prompt,output,latency_s
0,Write a tweet (<=200 chars) about reproducible...,Write a tweet (<=200 chars) about reproducible...,4.28
1,One sentence: why eval metrics matter beyond a...,One sentence: why eval metrics matter beyond a...,3.76
2,List 3 checks before deploying a model to prod...,List 3 checks before deploying a model to prod...,3.71
3,Explain temperature vs. top-p to a PM.,Explain temperature vs. top-p to a PM.\n\nTo a...,4.24


In [48]:
# 8b) Save to CSV (download from left sidebar in Colab)
out_path = "/mnt/data/hf_llm_batch_outputs.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)

Saved to: /mnt/data/hf_llm_batch_outputs.csv


## Ethics & safe use
- Verify critical facts (hallucinations happen).
- Respect privacy & licenses; avoid PHI/PII in prompts.
- Add guardrails/monitoring for production use.